In [2]:
import pandas as pd

# 모든 열(Columns) 다 보기
pd.set_option('display.max_columns', None)

# 모든 행(Rows) 다 보기
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)  # 또는 원하는 최대값


df = pd.read_csv("/data/ephemeral/home/work/python/upstageailab-nlp-summarization-nlp_3/data/raw/train.csv")

df.tail()

#df.value_counts('topic')
df['summary'].str.len().max()


# 1. 최대 길이 구하기
maxlen = df['summary'].str.len().max()

# 2. 최대 길이 row(혹은 여러 개)를 출력
max_rows = df[df['summary'].str.len() == maxlen]
max_rows

char_lengths = df['dialogue'].str.len()
print(f'최대 길이: {char_lengths.max()}')
print(f'평균 길이: {char_lengths.mean():.2f}')
print(f'중간값: {char_lengths.median()}')

char_lengths = df['summary'].str.len()
print(f'최대 길이: {char_lengths.max()}')
print(f'평균 길이: {char_lengths.mean():.2f}')
print(f'중간값: {char_lengths.median()}')

#dialogue: 2165, summary:376

최대 길이: 2165
평균 길이: 406.08
중간값: 369.0
최대 길이: 376
평균 길이: 85.79
중간값: 80.0


In [3]:

# ✅ 1. 기본 라이브러리 불러오기
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from transformers import AutoTokenizer

In [4]:
# ✅ 3. 데이터 로드 (src/data)
train_path = r"/data/ephemeral/home/work/python/upstageailab-nlp-summarization-nlp_3/data/raw/train.csv"
test_path = r"/data/ephemeral/home/work/python/upstageailab-nlp-summarization-nlp_3/data/raw/test.csv"

train = pd.read_csv(train_path, encoding="utf-8")  # or cp949 if needed
test = pd.read_csv(test_path, encoding="utf-8")

In [5]:
# ✅ 4. 기본 정보 확인
print("✅ train.csv shape:", train.shape)
print("✅ test.csv shape:", test.shape)
print("\n📌 train 컬럼 목록:", train.columns.tolist())
print("\n📌 train 샘플:\n", train.head())


✅ train.csv shape: (12457, 4)
✅ test.csv shape: (499, 2)

📌 train 컬럼 목록: ['fname', 'dialogue', 'summary', 'topic']

📌 train 샘플:
      fname  \
0  train_0   
1  train_1   
2  train_2   
3  train_3   
4  train_4   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    dialogue  \
0  #Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? \n#Person2#: 건강검진을 받으려고 왔어요. \n#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. \n#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. \n#Person1#: 음, 심각한 질병을 피하려면

In [6]:
# ✅ 5. 결측치 & 중복 확인
print("\n📌 결측치:\n", train.isnull().sum())
print("\n📌 중복 행 개수:", train.duplicated().sum())



📌 결측치:
 fname       0
dialogue    0
summary     0
topic       0
dtype: int64

📌 중복 행 개수: 0


In [7]:
# ✅ 6. 텍스트 길이 통계 분석
train['text_length'] = train['dialogue'].apply(len)
print("\n📌 text 길이 통계:\n", train['text_length'].describe())

print("📌 토픽별 샘플 텍스트 (최대 5개까지 출력):")
unique_topics = sorted(train["topic"].unique())[:5]  # 너무 많으면 제한

for topic in unique_topics:
    sample_text = train[train["topic"] == topic]["dialogue"].values[0][:300]
    print(f"\n🟡 [Topic: {topic}] 샘플 텍스트 (앞 300자):\n{sample_text}")



📌 text 길이 통계:
 count    12457.000000
mean       406.083487
std        197.566083
min         84.000000
25%        280.000000
50%        369.000000
75%        500.000000
max       2165.000000
Name: text_length, dtype: float64
📌 토픽별 샘플 텍스트 (최대 5개까지 출력):

🟡 [Topic: 'The Power of One' 책 읽기] 샘플 텍스트 (앞 300자):
#Person1#: 무슨 책 읽고 있어? 어젯밤부터 그 책에서 눈을 떼지 않더라.
#Person2#: 이 책 정말 재밌어. 도저히 손에서 뗄 수가 없어! 'The Power of One'이라고 해.
#Person1#: 나도 그렇게 재밌는 책 읽은 지 오래됐어.
#Person2#: 나도 그래. 첫 페이지부터 이 작가가 날 확 끌어들이더라고!
#Person1#: 네가 다 읽고 나면 나도 볼 수 있어?
#Person2#: 응, 지금 막 마지막 장 읽고 있는데 진짜 손에 땀을 쥐게 돼.
#Person1#: 아무 얘기도 하지 마. 나 혼자 

🟡 [Topic: 100만 달러 사용 계획] 샘플 텍스트 (앞 300자):
#Person1#: 너 만약에 100만 달러 받으면 뭐 할 거야?
#Person2#: 음, 잘 모르겠어. 아마 차를 사지 않을까?
#Person1#: 그거 좀 재미없어 보이는데.
#Person2#: 그럼 넌 뭐 할 건데?
#Person1#: 난 세계 일주 여행 갈 거야.
#Person2#: 어디로 갈 건데?
#Person1#: 첫 번째로 파리에 갈 거야. 파리에서 쇼핑하는 게 늘 꿈이었거든.
#Person2#: 좋은 생각이네. 거기서 돈 좀 쓰면 되겠다.
#Person1#: 응, 그리고 나서 로마에 갈 거야. 내가 이탈리아 음식을 

🟡 [Topic: 13일 금요일 미신] 샘플 텍스트 (앞 300자):
#Person1#

In [52]:
# ✅ 7. 스페셜 토큰
import re
import ast

def reg_masking(text):
  pattern = r"#\w+#"  # ## 사이의 값을 추출하는 정규식 패턴
  masked = re.findall(pattern, text)
  return masked

train_set = train['dialogue'].apply(lambda x:str(set(reg_masking(x))))
set_train = train_set.apply(ast.literal_eval)

test_set = test['dialogue'].apply(lambda x:str(set(reg_masking(x))))
set_test = test_set.apply(ast.literal_eval)


# 모든 set을 하나로 합치기
all_elements =  set().union(*set_train).union(*set_test)
print(all_elements)


{'#Kristin#', '#CardNumber#', '#PersonName#', '#DateOfBirth#', '#SSN#', '#Person7#', '#PhoneNumber#', '#Email#', '#Person4#', '#PassportNumber#', '#Alex#', '#Person1#', '#Address#', '#Person6#', '#Name#', '#Person3#', '#Price#', '#CarNumber#', '#Bob#', '#Person5#', '#Liliana#', '#Person2#'}


In [ ]:

# ✅ 1. 텍스트 길이 분포 시각화
plt.figure(figsize=(10, 6))
sns.histplot(train['text_length'], bins=50, kde=True, color='royalblue')
plt.title("텍스트 길이 분포", fontsize=16)
plt.xlabel("문자 수", fontsize=13)
plt.ylabel("샘플 수", fontsize=13)
plt.tight_layout()
plt.grid(True)
plt.show()

# ✅ 2. 요약 텍스트 길이 분포 (summary)
train['summary_len'] = train['summary'].apply(len)
plt.figure(figsize=(10, 6))
sns.histplot(train['summary_len'], bins=50, kde=True, color='salmon')
plt.title("요약 텍스트 길이 분포")
plt.xlabel("문자 수")
plt.ylabel("샘플 수")
plt.tight_layout()
plt.savefig("summary_length_distribution.png", dpi=200)
plt.close()

# ✅ 3. 레이블 분포 (상위 100개)
top_topics = train['topic'].value_counts().nlargest(100)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_topics.index, y=top_topics.values, palette="Set3")
plt.title("Topic 상위 100개 분포")
plt.xticks(rotation=75)
plt.tight_layout()
plt.savefig("top100_topic_distribution.png", dpi=200)
plt.close()

# ✅ 4. 고유 토픽 개수 출력
print("고유 Topic 개수:", train['topic'].nunique())

# ✅ 5. 특수문자/숫자 포함 비율
import re

def count_special_ratio(text):
    return len(re.findall(r"[^\w\s]", text)) / len(text)

train['special_ratio'] = train['dialogue'].apply(count_special_ratio)
print("평균 특수문자 비율:", train['special_ratio'].mean())

# ✅ 6. tokenizer 기준 토큰 길이
tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart") # 요약 task에 적합
train['dialogue_token_len'] = train['dialogue'].apply(lambda x: len(tokenizer.encode(x, truncation=False)))
train['summary_token_len'] = train['summary'].apply(lambda x: len(tokenizer.encode(x, truncation=False)))

plt.figure(figsize=(10, 5))
sns.histplot(train['dialogue_token_len'], color="dodgerblue", label="입력")
sns.histplot(train['summary_token_len'], color="tomato", label="요약")
plt.title("Token 길이 분포")
plt.xlabel("Token 수")
plt.legend()
plt.tight_layout()
plt.savefig("token_length_comparison.png", dpi=200)
plt.close()

# 요약 vs 입력 길이 비율 분포
# → 평균값이 너무 높거나 낮으면 모델이 잘 못 배움
#→ 보통 3~8 사이 분포가 안정적
train['dialogue_len'] = train['dialogue'].apply(len)
train['summary_len'] = train['summary'].apply(len)
train['length_ratio'] = train['dialogue_len'] / train['summary_len']

plt.figure(figsize=(10, 6))
sns.histplot(train['length_ratio'], bins=50, kde=True, color="purple")
plt.title("입력 길이 / 요약 길이 비율 분포", fontsize=16)
plt.xlabel("비율 (dialogue / summary)")
plt.ylabel("샘플 수")
plt.tight_layout()
plt.savefig("dialogue_summary_ratio.png", dpi=200)
plt.close()

# tokenizer 적용 후 입력/요약 토큰 수
# → max_seq_len, max_target_len 설정 근거가 됨
# → ex) 95% quantile 기준으로 자르는 게 좋음
tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart")

train['input_token_len'] = train['dialogue'].apply(lambda x: len(tokenizer.encode(x, truncation=False)))
train['output_token_len'] = train['summary'].apply(lambda x: len(tokenizer.encode(x, truncation=False)))

plt.figure(figsize=(10, 6))
sns.histplot(train['input_token_len'], color="dodgerblue", label="입력", kde=True)
sns.histplot(train['output_token_len'], color="tomato", label="요약", kde=True)
plt.title("Token 길이 분포 (tokenizer 적용 후)", fontsize=16)
plt.xlabel("토큰 수")
plt.legend()
plt.tight_layout()
plt.savefig("token_length_distribution.png", dpi=200)
plt.close()

# topic별 요약 길이 차이 확인
plt.figure(figsize=(14, 6))
sns.boxplot(x='topic', y='summary_len', data=train[train['topic'].isin(train['topic'].value_counts().nlargest(10).index)])
plt.title("상위 10개 Topic별 요약 길이 분포", fontsize=16)
plt.xticks(rotation=60)
plt.tight_layout()
plt.savefig("summary_length_per_topic.png", dpi=200)
plt.close()

# 중복 대화 탐지
duplicate_dialogues = train['dialogue'].duplicated().sum()
print(f"📌 중복된 dialogue 수: {duplicate_dialogues}개 ({duplicate_dialogues/train.shape[0]*100:.2f}%)")

In [24]:
import pandas as pd

# 모든 열(Columns) 다 보기
pd.set_option('display.max_columns', None)

# 모든 행(Rows) 다 보기
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)  # 또는 원하는 최대값

df.topic.unique()

array(['건강검진', '백신 접종', '열쇠 분실', ..., '작문 작성', '호텔 안내', '실직과 대처'],
      shape=(9235,), dtype=object)